In [1]:
from learning_pipelines import get_learning_results, get_hyper_param_results
import sklearn.svm as svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

from classification.keras_neural_nets import *

import os

/home/franciscosargo/.virtualenvs/pre_epi_seizures/local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import pylatex

In [3]:
import glob
import shutil
import os

In [4]:
from evaluation_tables import *

In [5]:
from classification.hp_opt.single_param import *

In [6]:
from learning_pipelines import load_eda

In [7]:
def create_tables_patient_specific(table):
        # Overal Patient Classification
        per_patient = table[['precision',
                             'recall',
                             'f1-score',
                             'patient_nr',
                             'Best Parameters',
                             'Labels']].groupby(['patient_nr', 'Labels']).describe()

        per_patient.loc[:, (['precision', 'recall', 'f1-score'], ['mean', 'std'])]

        # Overal Patient Classification
        per_patient = table[['precision',
                             'recall',
                             'f1-score',
                             'patient_nr',
                             'Best Parameters',
                             'Labels']].groupby(['Best Parameters','patient_nr', 'Labels']).describe()

        per_patient.loc[:, (['precision', 'recall', 'f1-score'], ['mean', 'std'])]
        
        print table
    

In [8]:
# Set the storage parameters 
disk = '/mnt/Seagate/pre_epi_seizures/'

In [9]:
# Set the latex report folder
latex_dir = 'latex/'

In [10]:
patient_lists = [[3], [4], [5], [8]]

In [11]:
# Allocate the feature set
feature_set = [('pca_beat_amp_computation', 0), ('hrv_time_features', 0)]

In [12]:
model_set = [
             (Pipeline([('KNN', KNeighborsClassifier())]), [{'KNN__n_neighbors': [i for i in xrange(1, 15, 2)]}]),
             (Pipeline([('GaussNB', GaussianNB())]), [{'GaussNB__priors': [None]}]),
             (Pipeline([('SVC', svm.SVC())]), [{'SVC__C': [2**i for i in xrange(-5, 11)], 
                                                'SVC__gamma':[2**i for i in xrange(-15, 1)]}])]


                                               

In [13]:
scaler_set = [StandardScaler()]

In [14]:
# Labeling structure
label_struct = {
                'inter_ictal':{
                    'label': 'Inter-Ictal Data Points',
                    'color': 'blue',
                    'intervals_samples': [(1000 * 0 * 60, 1000 * 20 * 60 )]
                             },
                'pre_ictal':{
                    'label': 'Pre-Ictal data points',
                    'color': 'yellow',
                    'intervals_samples': [(1000 * 20 * 60, 1000 * 49 * 60 )]
                             },
                'ictal':{
                    'label': 'Ictal data points',
                    'color': 'red',
                    'intervals_samples': [(1000 * 50 * 60, 1000 * 55 * 60 )]
                         },
                        }
baseline_label_struct = {
                 'baseline':{
                    'label': 'Baseline Data Points',
                    'color': 'green',
                    'intervals_samples': [(0, 1000 * 30 * 60 )]
                            },
                        }

plot_eda = False
learn_flag = False
compute_all_new = False


In [15]:
# Open latex document
eda_folder = 'EDanalysis_new/'
results_doc = pylatex.Document(default_filepath='/home/franciscosargo/results')
#results_doc = pylatex.Document(default_filepath=disk + eda_folder + 'results')


In [16]:
results_doc

Document('/home/franciscosargo/results', [Command(u'normalsize', Arguments(), Options())])

In [17]:
# Patient Specific results

for feature_slot, hyper_param in feature_set:
    
    print feature_slot
    print hyper_param
    
    with results_doc.create(pylatex.Section('The fancy stuff')):
    
        # Entry text, needs to be mended and corrected
        results_doc.append('The following results concern the feature set: ' + feature_slot)

        for patient_list in patient_lists:

            # Loop for scaler
            #for scaler in scaler_set:

            # Do some text
            #print 'Patient Number is' + str(patient_list)
            #print 'The Scaler is' + str(scaler)

            #print 'Here is a visualization of the normalized data'

            #print patient_list

            # EDA plots
            for scaler in scaler_set:

                # Loop for model
                for model in model_set:

                        # Unpack the variables of analysis
                        scaler = model[0]
                        pipe = model[1]
                        param_grid = model[2]


                        print 'The Hyper-Parameter Grid for optimization is:' + str(param_grid)

                        # Get table from original analysis
                        table = get_learning_results(label_struct, baseline_label_struct,
                                                      pipe, scaler, param_grid,
                                                      patient_list,
                                                      feature_slot,
                                                      hyper_param,
                                                      plot_eda,
                                                      learn_flag,
                                                      compute_all_new)

                        # Compute the statistics from the overall labels
                        per_label = evaluate_overall_labels(table)

                        # Compute the statistics for seizure type
                        per_type_of_seizure = evaluate_per_types_of_seizure(table)

                        # Get model name
                        model_name = pipe.steps[0][0]

                        # Get Hyper-parameterization results
                        cv_results=get_hyper_param_results(label_struct, baseline_label_struct,
                                                  pipe, scaler, param_grid,
                                                  patient_list,
                                                  feature_slot,
                                                  hyper_param,
                                                  plot_eda,
                                                  learn_flag,
                                                  compute_all_new)

                        param_name = param_grid[0].keys()[0]
                        single_param_var = param_grid[0][param_name]

                        single_mean_std_param_plot('fgjasljgf', 
                                                   cv_results, 
                                                   param_name, 
                                                   single_param_var,
                                                   feature_slot)

                        # Plot and save EDA



                        # Plot hyper-parameters function

                        single_mean_std_param_plot('fgjasljgf', 
                                       cv_results, 
                                       param_name, 
                                       single_param_var,
                                       feature_slot)

                        # 


                        # Get param

                        print per_label
                        print per_type_of_seizure




    
    


pca_beat_amp_computation
0


IndexError: tuple index out of range